In [14]:
class Value():
    def __init__(self, data):
        self.data = data

    def __repr__(self):
        return str(self.data)

    def __add__(self, other):
        return Value(self.data + other.data)

    def __mul__(self, other):
        return Value(self.data * other.data)

In [15]:
a = Value(10)
b = Value(20)
a * b

200